# Action Discretization comparison

This notebook compares the performance (calculation time, turns) of various levels of granularity when discretizing the action space.

In [1]:
from pod.board import PodBoard
from pod.ai.tree_search_controller import TreeSearchController
from pod.ai.rewards import re_dc
from pod.drawer import Drawer
from pod.ai.action_discretizer import ActionDiscretizer
from time import perf_counter

In [2]:
results = {}

for ad in [
    ActionDiscretizer(2,2),
    ActionDiscretizer(2,3),
    ActionDiscretizer(5,3),
    ActionDiscretizer(2,9),
    ActionDiscretizer(5,9)
]:
    print('========================= {} ========================='.format(ad))
    ad_res = {}
    for (board, name) in [(PodBoard.grid(), 'grid'), (PodBoard.tester(), 'tester'), (PodBoard.circle(), 'circle')]:
        print('   ---> ' + name)
        d = Drawer(board, controllers=[TreeSearchController(board, re_dc, 3, ad)])
        start = perf_counter()
        d.record(max_laps=2, max_turns=400)
        end = perf_counter()
        ad_res[name] = {
            'time': end - start,
            'turns': len(d.hist)
        }
        
    results[str(ad)] = ad_res


========================= AD(thr=2 ang=2) =========================
   ---> grid


Label(value='')

286 turns generated in 0.371 seconds
   ---> tester


Label(value='')

191 turns generated in 0.462 seconds
   ---> circle


...

288 turns generated in 467.353 seconds
   ---> tester


Label(value='')

400 turns generated in 648.009 seconds
   ---> circle


Label(value='')

93 turns generated in 147.006 seconds


In [7]:
from IPython.display import HTML, display

boardnames = list(results.items())[0][1].keys()
h = '<table style="border: solid">'

h += '<tr>'
h += '<td style="border-right: solid">Layout</td>'
for bn  in boardnames:
    h += '<th colspan="3" style="text-align: center; border-right: solid">{}</th>'.format(bn)
h += '<th colspan="3" style="text-align: center;">Total</th>'
h += '</tr>'

h += '<tr style="border-bottom: solid">'
h += '<td style="border-right: solid">Discretization</td>'
for bn in boardnames:
    h += '<th>turns</th><th>time</th><th style="border-right: solid">avg</th>'
h += '<th>turns</th><th>time</th><th>avg</th>'
h += '</tr>'

for (ad, data) in results.items():
    h += '<tr><th style="border-right: solid">{}</th>'.format(ad)
    totalTurns = 0
    totalTime = 0
    for bn in boardnames:
        h += '<td>%i</td>' % (data[bn]['turns'])
        h += '<td>%.3f</td>' % (data[bn]['time'])
        h += '<td style="border-right: solid">%.3f</td>' % (data[bn]['time'] / data[bn]['turns'])
        totalTurns += data[bn]['turns']
        totalTime += data[bn]['time']
    h += '<td>%i</td><td>%.3f</td><td>%.3f</td></tr>' % (totalTurns, totalTime, totalTime / totalTurns)

h += '</table>'

display(HTML(h))